In [1]:
import numpy as np;
import pandas as pd;
from numpy import savetxt

## Initialize Parameters

In [2]:
## parameters and global variables

district = "all"

zetaPanchayat = 0.1
zetaMunicipality = 0.2
zetaCorporation = 0.3


muForSmall = 0.09
muForMedium = 0.04
muForLarge = 0.02



## Load and Initialize Data

In [3]:
## Load population data
initDataDF = pd.read_csv("../data/" + district + "_population_data.csv")


## Load distance data
distanceData = pd.read_csv("../data/" + district + "_distance_matrix.csv")

## Load area data
areaData = pd.read_csv("../data/" + district + "_area_data.csv")

## Number of regions
r = len(initDataDF.index)

## Sort population data and distance data by name for easy matrix transformation
initDataDF.sort_values(by=['name'], inplace=True)
initDataDF.reset_index(drop=True, inplace=True)

distanceData.sort_values(by=['fromName', 'toName'], inplace=True)
distanceData.reset_index(drop=True, inplace=True)

areaData.sort_values(by=['name'], inplace=True)
areaData.reset_index(drop=True,inplace=True)

## Initialize Job 
initDataDF.loc[initDataDF['type']=='P','J'] = initDataDF[initDataDF['type']=='P']['N']*zetaPanchayat
initDataDF.loc[initDataDF['type']=='M','J'] = initDataDF[initDataDF['type']=='M']['N']*zetaMunicipality
initDataDF.loc[initDataDF['type']=='C','J'] = initDataDF[initDataDF['type']=='C']['N']*zetaCorporation

## Initialize T
areaData['areaSqKm'] = areaData['area']/(1000*1000)

initDataDF.loc[areaData['areaSqKm'] <= 25,'T'] = initDataDF[areaData['areaSqKm'] <= 25]['N']*muForSmall
initDataDF.loc[(areaData['areaSqKm'] > 25) & (areaData['areaSqKm'] <= 100),'T'] = initDataDF[(areaData['areaSqKm'] > 25) & (areaData['areaSqKm'] <= 100)]['N']*muForMedium
initDataDF.loc[areaData['areaSqKm'] > 100,'T'] = initDataDF[areaData['areaSqKm'] > 100]['N']*muForLarge

initDataDF.tail(15)


,name,type,N,J,T
1020,Vettikkavala__Kollam,P,36204,3620.4,1448.16
1021,Vettom__Malappuram,P,6577,657.7,591.93
1022,Vettoor__Thiruvananthapuram,P,18704,1870.4,1683.36
1023,Vijayapuram__Kottayam,P,30838,3083.8,2775.42
1024,Vilakudi__Kollam,P,32995,3299.5,2969.55
1025,Vilappil__Thiruvananthapuram,P,36212,3621.2,3259.08
1026,Vilavoorkal__Thiruvananthapuram,P,31761,3176.1,2858.49
1027,Vilayoor__Palakkad,P,23389,2338.9,2105.01
1028,Villiyappally__Kozhikode,P,34502,3450.2,3105.18
1029,Vithura__Thiruvananthapuram,P,26249,2624.9,524.98


## Matrix computation of TMatrix

### Prepare Distance matrix

In [4]:
distanceMatrix = np.array(distanceData['value'].to_numpy()).reshape(r,r)
np.fill_diagonal(distanceMatrix,1)
distanceMatrix


array([[     1,  62713, 257069, ...,  38737, 153934, 239018],
       [ 62713,      1, 194549, ...,  74778, 216606, 176575],
       [257069, 194549,      1, ..., 263797, 410410,  18277],
       ...,
       [ 38737,  74778, 263797, ...,      1, 159119, 246386],
       [153934, 216606, 410410, ..., 159119,      1, 392249],
       [239018, 176575,  18277, ..., 246386, 392249,      1]])

### Job Attraction Matrix

In [5]:
jobArray = initDataDF['J'].to_numpy()

jobByDSqMat = jobArray/np.multiply(distanceMatrix,distanceMatrix)
np.fill_diagonal(jobByDSqMat, 0)
jobAttractionMat = jobByDSqMat/jobByDSqMat.sum(axis=1, keepdims=True)
jobAttractionMat

array([[0.00000000e+00, 2.97481564e-04, 1.86261275e-05, ...,
        2.49687843e-04, 4.52717648e-05, 1.16420355e-05],
       [5.16512988e-05, 0.00000000e+00, 2.46031578e-05, ...,
        5.06907299e-05, 1.72974642e-05, 1.61383294e-05],
       [2.95880053e-06, 2.25093409e-05, 0.00000000e+00, ...,
        3.92062437e-06, 4.63773833e-06, 1.44985939e-03],
       ...,
       [5.37174760e-04, 6.28095317e-04, 5.30982637e-05, ...,
        0.00000000e+00, 1.27189274e-04, 3.28894127e-05],
       [4.49003504e-05, 9.88060539e-05, 2.89557908e-05, ...,
        5.86346846e-05, 0.00000000e+00, 1.71283590e-05],
       [3.80211001e-06, 3.03552033e-05, 2.98077044e-03, ...,
        4.99267937e-06, 5.64013125e-06, 0.00000000e+00]])

### Tij matrix

In [6]:
TiArray = initDataDF['T'].to_numpy().reshape(r,1)
TijMatrix = TiArray * jobAttractionMat

NiArray = initDataDF['N'].to_numpy()
TiiArray = NiArray - TijMatrix.sum(axis = 1)

TijMatrixDiag = np.diag(TiiArray)
TijMatrix += TijMatrixDiag
TijMatrix
TijMatrix.sum(axis=1)


array([ 7338., 31973., 33638., ..., 10239., 29316., 18176.])

## Save output

In [7]:
savetxt("../output/" + district + "_tmatrix_results.csv", TijMatrix, delimiter=',')

### Sparse CSV - on need basis

In [ ]:

data = []

for i in range(0,r):
    for j in range(0,r):
        Tij = TijMatrix[i][j]
        data.append({'iName': initDataDF.iloc[i]['name'], 'jName': initDataDF.iloc[j]['name'], 'Tij': Tij});
        
resultsDF = pd.DataFrame(data);

resultsDF.to_csv("../output/" + district + "_tmatrix_results.csv")
resultsDF


## Save top 5 

In [ ]:
resultsDF["rank"] = resultsDF.groupby(['iName'])['Tij'].rank("dense", ascending=False)
top5resultsDF = resultsDF[resultsDF["rank"] <= 6]
top5resultsDF["jName_Tij"] = top5resultsDF["jName"] + " (" + top5resultsDF["Tij"].astype(str)  + ")"
top5resultsDFPivot = top5resultsDF.pivot(index="iName", columns="rank", values="jName_Tij")
top5resultsDFPivot.to_csv("../output/" + district + "_tmatrix_top5_regions.csv")
